In [ ]:
import pandas as pd
import numpy as np
import cupy as cp
from scipy.special import stdtr
from sklearn.decomposition import PCA


In [ ]:
def qnorm_dataframe( data ):
    """
    quantile normalize a dataframe with numeric values only!
    Normalizes to rank mean
    Does not deal with ties
    """
    rank_mean = data.stack().groupby(data.rank(method='first').stack().astype(int)).mean()
    qnormed_data    = data.rank(method='min').stack().astype(int).map(rank_mean).unstack()
    return qnormed_data


def ut_as_list( dframe, diag=1, cols=['Row','Column','Value'] ):
  """
  for a symmetric dataframe, where cols=rows, get the upper triangle as a list of row/column pairs
  diag = 1 (default): ignore diagonal
  diag = 0: include diagonal
  """
  #if (dframe.index.name == dframe.columns.name):
  dframe.index.name = cols[0]
  dframe.columns.name = cols[1]
  #             dframe.index.name = dframe.index.name + '.1'
  #             dframe.index.name = dframe.index.name + '.2'
  d = dframe.where( np.triu( np.ones( dframe.shape ), k=diag).astype(bool))
  d = d.stack().reset_index()
  d.columns=cols
  return d

def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)

def PCA_whitening(X):
    centered_X = X - np.mean(X, axis = 0)
    cov = np.cov(centered_X.T)
    # Calculate the eigenvalues and eigenvectors of the covariance matrix
    eigVals, eigVecs = np.linalg.eig(cov)
    # Apply the eigenvectors to X
    transf_x = centered_X @ eigVecs
    whitened_x = transf_x / np.sqrt(eigVals + 1e-5)
    return whitened_x

def cholesky_whitening(df):
    cholsigmainv = np.linalg.cholesky(np.linalg.pinv(np.cov(df.T)))
    warped_screens = df.values @ cholsigmainv
    df_chol = pd.DataFrame(warped_screens,index=df.index.values,columns=df.columns.values)
    return df_chol

In [ ]:
# Z-scores
data=pd.read_csv('Zscores_dataframe.csv',header=0,index_col=0)
print(data.shape)

# Bayes Factors
# data=pd.read_csv('BF_dataframe.csv',header=0,index_col=0)
# print(data.shape)

# Ceres
# data= pd.read_csv('Ceres_dataframe.csv',header=0,index_col=0)
# print(data.shape)

# Chronos
# data= pd.read_csv('Chronos_dataframe.csv',header=0,index_col=0)
# print(data.shape)


In [ ]:
print(np.any(np.isnan(data)))
print(is_pos_def(np.cov(data.T)))

In [ ]:
# Quantile Normalization

data = qnorm_dataframe( data )

In [ ]:
# Boyle PCA

olfactory_genes = pd.read_csv('olfactory_genes.txt', header=None, squeeze=True)
olfactory_data = data.reindex(olfactory_genes).dropna()

transformation = PCA(n_components=4)
transformation.fit(olfactory_data)

top_PC_effects = transformation.inverse_transform(transformation.transform(data))

data -= top_PC_effects
data = data.iloc[:, :-4]

In [ ]:
# Whitening Covariance Transformation

data=PCA_whitening(data)

In [ ]:
# Cholesky Covariance Transformation

data=cholesky_whitening(data)

In [ ]:
# # GLS  *as in Wainberg et al, 2021 paper
# # with Cholesky

# cholsigmainv = np.linalg.cholesky(np.linalg.pinv(np.cov(data.T)))
# warped_screens = data.values @ cholsigmainv
# warped_intercept = cholsigmainv.sum(axis=0)

# GLS_coef = np.empty((len(warped_screens), len(warped_screens)))
# GLS_se = np.empty((len(warped_screens), len(warped_screens)))
# ys = np.array(warped_screens.T)

# start=timer()
# for gene_index in range(len(warped_screens)):
        
#     X = np.stack((warped_intercept,warped_screens[gene_index]), axis=1)
        
#     coef, residues = np.linalg.lstsq(X, ys, rcond=None)[:2]
        
#     df = warped_screens.shape[1] - 2
        
#     GLS_coef[gene_index] = coef[1]
        
#     GLS_se[gene_index] = \
#         np.sqrt(np.linalg.pinv(X.T @ X)[1, 1] * residues / df)
    
# time1=timer()-start
# print(time1)


# df = warped_screens.shape[1] - 2

# GLS_p = 2 * stdtr(df, -np.abs(GLS_coef / GLS_se))

# np.fill_diagonal(GLS_p, 1)

# GLS_logp=np.negative(np.log10(GLS_p))

# GLS_logp_df = pd.DataFrame(GLS_logp , index=data.index.values, columns=data.index.values)


In [ ]:
# OLS 

screen=data.values
intercept=np.ones(screen.shape[1],dtype=int)

GLS_coef = np.empty((len(screen), len(screen)))
GLS_se = np.empty((len(screen), len(screen)))
ys = np.array(screen.T)

start=timer()
for gene_index in range(len(screen)):
        
    X = np.stack((intercept,screen[gene_index]), axis=1)
        
    coef, residues = np.linalg.lstsq(X, ys, rcond=None)[:2]
        
    df = screen.shape[1] - 2
        
    GLS_coef[gene_index] = coef[1]
        
    GLS_se[gene_index] = \
        np.sqrt(np.linalg.pinv(X.T @ X)[1, 1] * residues / df)
    
time1=timer()-start
print(time1)

df = screen.shape[1] - 2

GLS_p = 2 * stdtr(df, -np.abs(GLS_coef / GLS_se))

np.fill_diagonal(GLS_p, 1)

GLS_logp=np.negative(np.log10(GLS_p))

GLS_logp_df = pd.DataFrame(GLS_logp , index=data.index.values, columns=data.index.values)

In [ ]:
# Create OLS Pairs

OLS_pairs = ut_as_list(GLS_logp_df, diag=1, cols=['Gene1','Gene2','OLS logP'] ).sort_values('OLS logP', ascending=False).head(100000) 


In [ ]:
# PCC 

# #  Correlation matrix
Corr_df = pd.DataFrame( np.corrcoef(data.values) , index=data.index.values, columns=data.index.values)

# # Create PCC Pairs
PCC_pairs = ut_as_list(Corr_df,cols=['Gene1','Gene2','PCC']).sort_values(by='PCC',key=abs, ascending=False).head(100000)

In [ ]:
# Save Pairs

# PCC_pairs.to_csv(r'',index=False)

# OLS_pairs.to_csv(r'',index=False)

